In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI XGBoost超参数调整

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fhyperparameter_tuning_xgboost.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/hyperparameter_tuning_xgboost.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 查看GitHub上的页面
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用XGBoost进行Vertex AI超参数调整。

了解更多关于[Vertex AI超参数调整](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

### 目标

在本教程中，您将学习如何使用**Vertex AI超参数调整**服务来训练一个XGBoost模型。

本教程使用以下Vertex AI服务：

- **Vertex AI训练**
- **Vertex AI超参数调整**（使用**Vertex AI Vizier**）

执行的步骤包括：

- 使用Python训练应用程序包进行训练。
- 在超参数调整过程中报告准确性。
- 使用GCSFuse将模型工件保存到Cloud Storage。
- 列出最佳模型。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris) 的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview)。该数据集不需要任何特征工程。本教程中的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测出三种类型的鸢尾花品种：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

### 成本

本教程使用 Google 云的收费组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing) ，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

开始吧

### 安装 Vertex AI SDK for Python

In [ ]:
# install packages
! pip3 install --upgrade --quiet google-cloud-aiplatform

### 重新启动运行时（仅适用于Colab）

为了使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续进行下一步之前，请等待完成。⚠️</b>
</div>

### 在谷歌Colab上验证您的笔记本环境

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息

了解有关[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的更多信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产品，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在，则运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

设置硬件加速器

你可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，为每个VM分配4个Nvidia Telsa T4 GPU，你可以指定：

（aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4）

否则，请指定`(None, None)`以使用在CPU上运行的容器映像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

**注意**：在GPU支持方面，在TF版本2.3之前发布的版本在加载本教程中的自定义模型时会失败。此问题在TF版本2.3及以上中得到修复。这是由于在服务函数中生成的静态图运算。如果在您自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3的容器映像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建Docker容器映像。


有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

#### 设置机器类型

设置变量 `TRAIN_COMPUTE` 来配置用于训练的虚拟机的计算资源。了解更多关于[用于训练支持的机器类型](https://cloud.google.com/vertex-ai/docs/training/configure-compute#machine-types)。

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

## Python 培训应用程序包

在这个示例中，您将使用 Vertex AI 超参数调优服务，并使用执行 Python 培训应用程序包的培训作业。

了解更多关于[Vertex AI 中的超参数调优](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

看一看如何为在 Vertex AI 中运行自定义训练作业而构建 Python 包的结构。该包包含以下目录结构：

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 *setup.cfg* 和 *setup.py* 提供了将包安装到 Docker 镜像的操作环境中的说明。

文件 *trainer/task.py* 是在运行自定义训练作业时执行的 Python 脚本。

### 创建一个文件夹结构作为Python包

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 创建一个训练脚本

接下来，您要为训练工作创建一个名为*task.py*的脚本。一些值得注意的步骤包括：

1. <u>处理命令行参数：</u>
    - `model-dir`：保存训练模型的位置。如果没有传递值，保存模型的位置将从环境变量`AIP_MODEL_DIR`获取，默认为临时存储桶位置。
    - `dataset_data_url`：要下载的训练数据的位置。
    - `dataset_labels_url`：要下载的训练标签的位置。
    - `boost-rounds`：可调的超参数。
1. <u>数据预处理（`get_data()`）：</u>
    - 下载数据集并拆分为训练集和测试集。
1. <u>训练（`train_model()`）：</u>
    - 训练模型。
1. <u>评估（`evaluate_model()`）：</u>
    - 评估模型。
    - 如果进行超参数调整，报告准确度指标。
1. <u>保存模型产物：</u>
    - 将模型产物和评估指标保存在由`model-dir`指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb
import hypertune
import argparse
import logging

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--dataset-data-url", dest="dataset_data_url",
                    type=str, help="Download url for the training data.")
parser.add_argument("--dataset-labels-url", dest="dataset_labels_url",
                    type=str, help="Download url for the training data labels.")
parser.add_argument("--boost-rounds", dest="boost_rounds",
                    default=20, type=int, help="Number of boosted rounds")
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def get_data():
    logging.info("Downloading training data and labelsfrom: {}, {}".format(args.dataset_data_url, args.dataset_labels_url))
    # gsutil outputs everything to stderr. Hence, the need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_data_url, 'data.csv'], stderr=sys.stdout)
    subprocess.check_call(['gsutil', 'cp', args.dataset_labels_url, 'labels.csv'], stderr=sys.stdout)


    # Load data into pandas, then use `.values` to get NumPy arrays
    data = pd.read_csv('data.csv').values
    labels = pd.read_csv('labels.csv').values

    # Convert one-column 2D array into 1D array for use with XGBoost
    labels = labels.reshape((labels.size,))

    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=7)

    # Load data into DMatrix object
    dtrain = xgb.DMatrix(train_data, label=train_labels)
    return dtrain, test_data, test_labels

def train_model(dtrain):
    logging.info("Start training ...")
    # Train XGBoost model
    model = xgb.train({}, dtrain, num_boost_round=args.boost_rounds)
    logging.info("Training completed")
    return model

def evaluate_model(model, test_data, test_labels):
    dtest = xgb.DMatrix(test_data)
    pred = model.predict(dtest)
    predictions = [round(value) for value in pred]
    # evaluate predictions
    accuracy = accuracy_score(test_labels, predictions)
    logging.info(f"Evaluation completed with model accuracy: {accuracy}")

    # report metric for hyperparameter tuning
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='accuracy',
        metric_value=accuracy
    )
    return accuracy


dtrain, test_data, test_labels = get_data()
model = train_model(dtrain)
accuracy = evaluate_model(model, test_data, test_labels)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Export the classifier to a file
gcs_model_path = os.path.join(args.model_dir, 'model.bst')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
model.save_model(gcs_model_path)

logging.info("Saving metrics to {}/metrics.json". format(args.model_dir))
gcs_metrics_path = os.path.join(args.model_dir, 'metrics.json')
with open(gcs_metrics_path, "w") as f:
    f.write(f"{'accuracy: {accuracy}'}")

### 将存储训练脚本在云存储桶

压缩整个训练文件夹，然后将其存储在云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

定义机器配置

定义您自定义超参数调整作业的机器配置。这告诉Vertex AI在作业启动时应提供哪种类型的机器实例。

您可以指定以下参数：
- `machine_type`：要分配的GCP实例的类型，例如n1-standard-8。
- `accelerator_type`：硬件加速器的类型（如果有的话）。在本教程中，如果您之前设置了变量`TRAIN_GPU`，则表示您正在使用GPU。否则，您正在使用CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

## 定义磁盘配置（可选）

可以选择性地为您的自定义超参数调整作业定义磁盘配置。这将告诉 Vertex AI 在超参数调整中的每个机器实例上提供什么类型和大小的磁盘。

您可以指定以下参数：
  - `boot_disk_type`: SSD 或 Standard。SSD 更快，而 Standard 更便宜。默认为 SSD。
  - `boot_disk_size_gb`: 磁盘大小（单位：GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 100  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

## 设置工作池规格

为自定义训练容器指定以下工作池规格：

- `args`：传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：指定在云存储桶中存储模型工件的位置。
  - `--dataset-data-url`：要下载的训练数据的位置。
  - `--dataset-labels-url`：要下载的训练标签的位置。
  - `--boost-rounds`：在训练XGBoost模型时设置可调整的超参数`num_boost_round`。

In [ ]:
# Set path to save model
MODEL_DIR = "{}/aiplatform-custom-job".format(BUCKET_URI)
# Set the source path to the dataset
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris"

# Set the command-line arguments
CMDARGS = [
    "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
    "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
]

# Set the worker pool specs
worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_iris.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

## 创建自定义训练任务

使用`CustomJob`类来创建具有以下参数的自定义训练任务：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：相应VM实例的规格。
- `base_output_dir`：用于存储模型制品的Cloud Storage位置。

In [ ]:
job = aiplatform.CustomJob(
    display_name="iris",
    worker_pool_specs=worker_pool_spec,
    base_output_dir=MODEL_DIR,
)

创建一个超参数调优作业

使用`HyperparameterTuningJob`类来创建一个具有以下参数的超参数调优作业：

- `display_name`：用于自定义作业的可读性名称。
- `custom_job`：为训练创建的CustomJob对象。来自此自定义作业的工作池规范适用于所有试验中创建的CustomJob。
- `metrics_spec`：要优化的指标。字典键是由您的训练作业报告的metric_id，字典值是该指标的优化目标（“最小化”或“最大化”）。
- `parameter_spec`：要优化的参数。字典键是作为命令行关键字参数传递到您的训练作业中的metric_id，字典值是指标的参数规范。
- `search_algorithm`：要使用的搜索算法。接受`grid`、`random`和`None`作为值。用于自定义训练的超参数调优使用[Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview)进行训练作业。
- `max_trial_count`：要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aiplatform.HyperparameterTuningJob(
    display_name="iris",
    custom_job=job,
    metric_spec={
        "accuracy": "maximize",
    },
    parameter_spec={
        "boost-rounds": hpt.IntegerParameterSpec(min=10, max=100, scale="linear"),
    },
    search_algorithm=None,
    max_trial_count=6,
    parallel_trial_count=1,
)

运行超参数调整作业

使用`run（）`方法执行超参数调整作业。

In [ ]:
hpt_job.run()

### 显示超参数调整作业试验结果

一旦超参数调整工作成功完成，您可以使用“trials”属性访问每个试验的结果。

In [ ]:
print(hpt_job.trials)

找到最佳试验

识别最佳试验并打印详细信息。

In [ ]:
# Initialize a tuple to identify the best configuration
best = (None, None, None, 0.0)
# Iterate through the trails and update the best configuration
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

# print details of the best configuration
print(best)

列出最佳模型

最佳模型的模型文件保存在： 

    MODEL_DIR/<best_trial_id>/model

In [ ]:
# Fetch the best model
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

! gsutil ls {BEST_MODEL_DIR}

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 超参数调整作业（HyperparameterTuning Job）
- 云存储桶（Cloud Storage bucket）
- 本地生成的文件

In [ ]:
# Delete the hyperparameter tuning job
hpt_job.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True to delete the bucket

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Delete the locally generated files
! rm -rf custom/
! rm custom.tar.gz